In [73]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torchaudio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

This model is going to be built on a CPU. Importing csv files for audio below.

In [74]:
csvData = pd.read_csv('./pd_speech_features.csv')
print(csvData.head()) #Use to access the data within it 

   id  gender      PPE      DFA     RPDE  numPulses  numPeriodsPulses  \
0   0       1  0.85247  0.71826  0.57227        240               239   
1   0       1  0.76686  0.69481  0.53966        234               233   
2   0       1  0.85083  0.67604  0.58982        232               231   
3   1       0  0.41121  0.79672  0.59257        178               177   
4   1       0  0.32790  0.79782  0.53028        236               235   

   meanPeriodPulses  stdDevPeriodPulses  locPctJitter  ...  \
0          0.008064            0.000087       0.00218  ...   
1          0.008258            0.000073       0.00195  ...   
2          0.008340            0.000060       0.00176  ...   
3          0.010858            0.000183       0.00419  ...   
4          0.008162            0.002669       0.00535  ...   

   tqwt_kurtosisValue_dec_28  tqwt_kurtosisValue_dec_29  \
0                     1.5620                     2.6445   
1                     1.5589                     3.6107   
2          

Create a *rapper class* for the data set using ```torch.utils.data.Dataset``` to load files and formal.

In [75]:
#coorelation of a matrix in and pulses. Visualizes it.
csvData[["numPulses","numPeriodsPulses"]].corr()
#Therefore because 2 attributes are extremely similar to each other, 
#discard numPeriodsPulses
del csvData['numPeriodsPulses']
#Make a heatmap of the data to find the best results
#sns.heatmap(csvData[csvData.columns[0:10]].corr(),annot=True)
del csvData['locPctJitter']  #coor of PCT Jitter and IoC ABS Jitter
del csvData['ddpJitter']     #coor ppq5jitter and ddpjitter
del csvData['locShimmer']    #coor OCShimmer and IOCDBShimmer
del csvData['apq11Shimmer']
del csvData['meanNoiseToHarmHarmonicity']
csvData.head()

#Reduce the number of rows by using automatic selections
csvData.shape


(756, 749)

In [77]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
coor_y = csvData['class']
coor_x = csvData.iloc[:,0:747]
xnew2=SelectKBest(f_classif, k=20).fit_transform(coor_x, coor_y)

In [78]:
X=pd.DataFrame(xnew2)
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,2.4874,0.028115,0.014642,0.019681,0.012829,0.021703,4.8840,-229943.2967,-201985.0408,0.000059,0.005199,0.004324,0.013373,-0.026321,-0.024286,-0.048924,0.022796,0.024286,0.048924,1.5466
1,2.8986,0.022166,0.025500,0.019374,0.010645,0.020296,4.8483,-230526.8175,-203389.4678,0.000169,0.005221,0.004344,0.011184,-0.070039,-0.099695,-0.112210,0.080975,0.099695,0.112210,1.5530
2,3.2208,0.032326,0.024607,0.024819,0.016553,0.023186,1.9849,-246592.6024,-214707.2576,0.000021,0.003136,0.002657,0.008093,-0.019435,-0.026241,-0.039886,0.013810,0.026241,0.039886,1.5399
3,3.1023,0.032577,0.017274,0.016129,0.015533,0.011976,3.2053,-255107.4287,-160775.5650,0.000493,0.004537,0.016104,0.051620,-0.121640,-0.129990,-0.268930,0.111290,0.122970,0.273530,6.9761
4,2.9451,0.031989,0.013791,0.016059,0.014382,0.012686,2.7170,-252647.9964,-154087.6641,0.000328,0.003931,0.015422,0.051298,-0.078138,-0.115900,-0.227210,0.086621,0.102950,0.235860,7.8832


In [81]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, coor_y, test_size=0.25, random_state=43421)
rdclass=RandomForestClassifier()
rdclass.fit(X_train,y_train)
ypred=rdclass.predict(X_test)

In [83]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,ypred))
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,ypred))

[[ 17  28]
 [  9 135]]
0.8042328042328042
